In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test_random_400-2.csv
/kaggle/input/test_top_cosine_200-2.csv
/kaggle/input/test_top_rougeL_200-2.csv
/kaggle/input/train_data.csv
/kaggle/input/test_random_600-2.csv


# Install

In [2]:
!pip install -q transformers accelerate rouge-score pandas

In [3]:
!pip install --upgrade transformers

# Prompt

In [4]:
EXTRACTOR_PROMPT = """
Your task is to shorten and extract only the introduction and description information from an app. You are given the following description for an app:
<Description>
$readme_text
</Description>
 
# Steps
- **Identify the structure of the app's description**: The app's description is a structure text file that might contains many sections such as introduction, description, key features, advantages, detailed use case, setup instructions, compatibility,...
- **Remove all sections that are not relevant to the app's description**: Irrelevant sections might include technical guidance (installing/running/specification... instruction), compatibility, troubleshooting,...
- **Remove all unnecessary links/tags**: Identify all links/tags that DO NOT contribute to the description of the app. You must remove all of these reference links and tags.
- **Return only text that is relevant to the description of the app**: The output should only contains the text that is relevant to the introduction/description of the app, including the app name/title, app feature description/purpose statement/overview. DO NOT include any output identifications such as: "Here's the ..." or "Extracted App's description:"
"""

INITIAL_SUMMARIZER_PROMPT = """
Summarize the following extracted text from an app's description into a short term/phrase introducing the app:
<EXTRACTED_APP'S_DESCRIPTION>
$extracted_text
</EXTRACTED_APP'S_DESCRIPTION>
 
The output should include only a short term/phrase introducing the app.
"""

BASE_TEMPLATE = """
Summarize the following extracted text from an app's description into a short term/phrase introducing the app:
<EXTRACTED_APP'S_DESCRIPTION>
$extracted_text
</EXTRACTED_APP'S_DESCRIPTION>

{generated_instruction}

The output should include only a short term/phrase introducing the app.
"""
TEACHER_PROMPT = """
You are a professional Prompt Engineer. You are working on a system using a Large Language Model (LLM) to help developers automatically generate a short Description term/phrase contain key concept/idea from an extracted text of the description of an app. Your task is to modify and improve the current prompt of the LLM based on the result of testing on a data include a description and a ground truth description.
 
# Steps:
- **Analyze the data for testing**: Analyze the following data include an extracted text from a description and a ground truth description from an app:
<EXTRACTED_TEXT>
$extracted_text
</EXTRACTED_TEXT>
 
<GROUND_TRUTH DESCRIPTION>
$description
</GROUND_TRUTH DESCRIPTION>
- **Review the current result**: Review the generated description using the extracted text its ROUGE score on the ground truth description to identify improvements that could be made:
<GENERATED_DESCRIPTION>
$generated_about
</GENERATED_DESCRIPTION>
<ROUGE_SCORE>
$rouge_score
</ROUGE_SCORE>
- **Prioritize extracting existing name/title, feature description/purpose statement/overview**: Compare the text from the beginning of the extracted text from description and the ground truth description. If the ground truth description is already existed in this extracted text as a name/title, feature description/purpose statement/overview, you must include in the new prompt the instruction to prioritize using it.
- **Modify the current prompt**: Identify mistakes and lacking instructions in the current prompt from the result of the above review. You should preserve the current prompt as much as possible and only make small changes to the prompt based on the identified mistakes and lacking instructions.
<CURRENT_PROMPT>
$summarizer_prompt
</CURRENT_PROMPT>
As the new prompt will not include the ground truth description, DO NOT mention about the ground truth description in the new prompt. DO NOT include any reasoning/explanation like "Based on the result of the above review:", "Here's the", ... or any output identifiers like "Prompt:", "New Prompt", ... The output should only include a string representing the new prompt for the LLM
"""

COMBINE_PROMPT = """
You are a professional Prompt Engineer. You are working on a system using a Large Language Model (LLM) to help developers automatically generate a short Description term/phrase contain key concept/idea from an extracted text of the description of an app. Your task is to combine several candidate prompts for the LLM into a final prompt.
 
# Steps:
- **Review all candidate prompts**: Analyze the following prompts to identify common parts to be included in the final prompt and also includes specific details or conditional key points from these prompts to be included in the final prompt
<CANDIDATE_PROMPTS>
$summarizer_list
</CANDIDATE_PROMPTS>
- **Generate a final prompt**: Based on the common parts and conditional key points, generate a final prompt for the LLM.

# Output Format:
Do not include any reasoning/explanation like "Based on the result of the above review:", "Here's the", ... or any output identifiers like "Prompt:", "New Prompt", ... The output should only include a string representing the prompt for the LLM
"""

ANALYSIS_PROMPT = """
A Large Language Model is used to auto-summarize app's description section and generate a concise description for an app. Your task is to analyze its result to provide a short advice on how to improve the generated description. You will be provided with the actual and the generated description section of an app and the ROUGE score between them:
<ACTUAL_DESCRIPTION>
$ground_truth
</ACTUAL_DESCRIPTION>
 
<GENERATED_DESCRIPTION>
$generated_about
</GENERATED_DESCRIPTION>
 
<ROUGE_SCORE>
$score
</ROUGE_SCORE>
 
# Steps:
- List the differences between the actual and the generated description section that results in the ROUGE score.
- Choose one main reason among the differences that best represents the ROUGE score.
- The output must be only one short advise on how to improve the generated description from the description base on that main reason.
# Output Format:
1 concise and short advice sentence
"""

ANALYSIS_SUMMARIZER_PROMPT = """
A Large Language Model is used to auto-summarize app's description section and generate a concise description for an app. You are given the following evaluating results on a dataset comparing app descriptions generated from a detail description by the Large Language Model and the actual descriptions:
<ANALYSIS_RESULT>
$analysis_result
</ANALYSIS_RESULT>
# Steps:
- Review the overall tendency of the analysis results.
- Summary one main point that best represents the analysis results.
- Give only one advise on how to improve the generated description.
# Output Format:
1 concise and short advice sentence
"""

SEQUENTIAL_TEACHER_PROMPT = """
You are a professional Prompt Engineer. You are using a Large Language Model (LLM) to help developers automatically generate a short Description term/phrase contain key concept/idea from the description of an app. The LLM include a prompt to extract the relevant information from the description and a prompt to generate a short description term/phrase contain key concept/idea. Your task is to optimize the current prompt to improve the performance of the LLM.
<CURRENT_PROMPT>
$current_summarizer_prompt
</CURRENT_PROMPT>
 
You should use the following advising analysis to help you optimize the prompts of the two agents:
<ANALYSIS_RESULT>
$current_analysis
</ANALYSIS_RESULT>
 
Here is an example of prompts that have good performance. FOLLOW this example to optimize the prompt of the LLM.
<GOOD_PROMPT>
$best_summarizer_prompt
</GOOD_PROMPT>
<SCORE>
$best_score
</SCORE>
<ADVISING_ANALYSIS>
$best_analysis_result
</ADVISING_ANALYSIS>
 
Here is an example of prompts that have bad performance. AVOID the mistakes of this example to optimize the prompt of the LLM.
<BAD_PROMPT>
$worst_summarizer_prompt
</BAD_PROMPT>
<SCORE>
$worst_score
</SCORE>
<ADVISING_ANALYSIS>
$worst_analysis_result
</ADVISING_ANALYSIS>
 
You should preserve the current prompt as much as possible and only make small changes to the prompt based on the advising analysis. You must include in the detail part that the description is "A shortest term or phrase include only the concept/idea of the app, without any explanations or details". The answer must only include the new prompt for the LLM
 
# Output Format:
Prompt: <Prompt>
"""

OPTIMIZED_SUMMARIZER_PROMPT = """"Summarize the following extracted text from an app's description into a short term/phrase introducing the app. If a name/title, feature description, purpose statement, or overview is present at the beginning of the extracted text, prioritize using it verbatim. Ensure the description captures the app's key concept and unique offerings, preserving terms like "Open Source" and "privacy-friendly" if they appear in the initial overview:

<EXTRACTED_APP'S_DESCRIPTION>
$extracted_text
</EXTRACTED_APP'S_DESCRIPTION>

The output should include only a short term/phrase introducing the app."""


# Rouge Score

In [5]:
from rouge_score import rouge_scorer

class ROUGE:
    def __init__(self):
        self.scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    def get_RougeL(self, string_1: str, string_2: str):
        score = self.scorer.score(string_1, string_2)

        return score["rougeL"].fmeasure

    def get_Rouge1(self, string_1: str, string_2: str):
        score = self.scorer.score(string_1, string_2)

        return score["rouge1"].fmeasure

    def get_Rouge2(self, string_1: str, string_2: str):
        score = self.scorer.score(string_1, string_2)

        return score["rouge2"].fmeasure

# Initiate model

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

class LocalLLMClient:
    def __init__(self, model_name: str, max_new_tokens: int = 100, temperature: float = 0.7):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            # torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            device_map="auto"
        )

        if self.tokenizer.pad_token is None or self.tokenizer.pad_token == self.tokenizer.eos_token:
            self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
            self.model.resize_token_embeddings(len(self.tokenizer))
        self.model.config.pad_token_id = self.tokenizer.pad_token_id

        self.max_new_tokens = max_new_tokens
        self.temperature = temperature

    def generate(self, prompt: str, temperature: float = None, max_new_tokens: int = None) -> str:
        temp = self.temperature if temperature is None else temperature
        max_tokens = self.max_new_tokens if max_new_tokens is None else max_new_tokens
        do_sample = temp > 0.0

        messages = [{"role": "user", "content": prompt}]
        inputs = self.tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        )
        input_ids = inputs.to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=input_ids,
                max_new_tokens=max_tokens,
                temperature=temp,
                do_sample=do_sample,
                pad_token_id=self.tokenizer.pad_token_id
            )

        result = self.tokenizer.decode(
            outputs[0][input_ids.shape[-1]:], skip_special_tokens=True
        )
        return result.strip()

# PromptCombineAgent

In [7]:
class PromptCombineAgent:
    def __init__(self,llm):
        self.llm = llm

    def _build_prompt(self, summarizer_list: str) -> str:
        prompt = Template(COMBINE_PROMPT)
        return prompt.substitute(summarizer_list=summarizer_list)

    def _parse_answer(self, answer: str) -> str:
        if "$extracted_text" not in answer:
            answer = re.sub(r"\btext\b", "$extracted_text", answer, count=1)
        return answer.strip()

    def _clean_prompt_list(self, prompt_list: list[str]) -> str:
        ret = ""
        for i, prompt in enumerate(prompt_list):
            ret += f"Extractor Prompt #{i + 1}:\n{prompt}\n\n"
        return ret

    def run(self, prompt_list: list[str]) -> str:
        summarizer_list = self._clean_prompt_list(prompt_list)
        full_prompt = self._build_prompt(summarizer_list)
        answer = self.llm.generate(full_prompt, temperature=0.2, max_new_tokens=120)
        print("Raw output from model:\n", answer)
        summarizer_prompt = self._parse_answer(answer)
        return summarizer_prompt


# TeacherAgent

In [8]:
class TeacherAgent:
    def __init__(self,llm):
        self.llm = llm

    def _build_prompt(
        self,
        extracted_text: str,
        description: str,
        generated_about: str,
        rouge_score: float,
        summarizer_prompt: str,
    ) -> str:
        prompt = Template(TEACHER_PROMPT)
        return prompt.substitute(
            extracted_text=extracted_text,
            description=description,
            generated_about=generated_about,
            rouge_score=rouge_score,
            summarizer_prompt=summarizer_prompt,
        )

    def _parse_answer(self, answer: str) -> str:
        if "$extracted_text" not in answer:
            answer = re.sub(r"\$\w*text\w*|\btext\b", "$extracted_text", answer, count=1)
        if "$extracted_text" not in answer:
            answer = f"{answer.strip()}\n\n$extracted_text"
        return answer.strip()

    def run(
        self,
        extracted_text: str,
        description: str,
        generated_about: str,
        rouge_score: float,
        summarizer_prompt: str,
    ) -> str:
        full_prompt = self._build_prompt(
            extracted_text, description, generated_about, rouge_score, summarizer_prompt
        )
        answer = self.llm.generate(full_prompt, temperature=0.7, max_new_tokens=120)
        print("Teacher raw answer:\n", answer)
        summarizer_prompt = self._parse_answer(answer)
        return summarizer_prompt

# SummarizerAgent

In [9]:
class SummarizerAgent:
    def __init__(self,llm):
        self.llm = llm

    def _build_prompt(self, prompt: str, extracted_text: str) -> str:
        return Template(prompt).safe_substitute(extracted_text=extracted_text)

    def run(self, prompt: str, extracted_text: str) -> str:
        full_prompt = self._build_prompt(prompt, extracted_text)
        return self.llm.generate(full_prompt, temperature=0.0, max_new_tokens=25)


# ExtractorAgent

In [10]:
class ExtractorAgent:
    def __init__(self,llm):
        self.llm = llm

    def _build_prompt(self, prompt: str, readme_text: str) -> str:
        return Template(prompt).substitute(readme_text=readme_text)

    def run(self, prompt: str, readme_text: str) -> str:
        full_prompt = self._build_prompt(prompt, readme_text)
        return self.llm.generate(full_prompt, temperature=0.0, max_new_tokens=150)

# Main flow

In [11]:
from datetime import datetime as dt

def get_current_time() -> float:
    """Get current time in UNIX timestamp"""
    return dt.now().timestamp()

class ParallelOptimizer:
    def __init__(self, extractor_agent, summarizer_agent, teacher_agent, prompt_combine_agent, threshold=0.7):
        self.extractor_agent = extractor_agent
        self.summarizer_agent = summarizer_agent
        self.teacher_agent = teacher_agent
        self.prompt_combine = prompt_combine_agent
        self.extractor_prompt = EXTRACTOR_PROMPT
        self.summarizer_prompt = INITIAL_SUMMARIZER_PROMPT
        self.debug_result = {}
        self.threshold = threshold

    def run(self, max_iterations: int, train_data: list[dict]):
        self.debug_result = {
            "timestamp": get_current_time(),
            "Teacher Prompt": TEACHER_PROMPT,
            "Combine Prompt": COMBINE_PROMPT,
            "Initial Extractor Prompt": self.extractor_prompt,
            "Initial Summarizer Prompt": self.summarizer_prompt,
        }

        iteration_debug = []
        data_prompt = []

        for i, data in enumerate(train_data):
            data_debug = []
            self.summarizer_prompt = INITIAL_SUMMARIZER_PROMPT

            description = data["description_short"]
            html = data["description_html_clean"]
            print(f"Data #{i}:\n- Description: {description}")

            best_score = 0
            best_summarizer_prompt = self.summarizer_prompt

            extracted_text = self.extractor_agent.run(
                prompt=self.extractor_prompt, readme_text=html
            )

            max_total_iter = max_iterations
            N = 3
            K = 2
            X = 2
            extend_step = 3

            no_improve_count = 0
            decrease_count = 0
            increase_streak = 0
            max_rougeL_so_far = 0
            iter = 0

            while iter < max_total_iter:
                print(f"Iteration #{iter}:")
                print(f"Extracted Text: {extracted_text}")

                about = self.summarizer_agent.run(
                    prompt=self.summarizer_prompt, extracted_text=extracted_text
                )

                print(f"Generated About: {about}")

                rougeL_score = ROUGE().get_RougeL(string_1=about, string_2=description)
                rouge1_score = ROUGE().get_Rouge1(string_1=about, string_2=description)
                rouge2_score = ROUGE().get_Rouge2(string_1=about, string_2=description)

                print(f"Rouge1 Score: {rouge1_score}")
                print(f"Rouge2 Score: {rouge2_score}")
                print(f"RougeL Score: {rougeL_score}")

                turn_debug = {
                    "summarizer_prompt": self.summarizer_prompt,
                    "description_html_clean": html,
                    "extracted_text": extracted_text,
                    "description_short": description,
                    "generated_about": about,
                    "rouge1_score": rouge1_score,
                    "rouge2_score": rouge2_score,
                    "rougeL_score": rougeL_score,
                }

                data_debug.append(turn_debug)

                if rougeL_score > best_score:
                    best_score = rougeL_score
                    best_summarizer_prompt = self.summarizer_prompt

                if rougeL_score > max_rougeL_so_far:
                    delta = rougeL_score - max_rougeL_so_far
                    max_rougeL_so_far = rougeL_score
                    no_improve_count = 0
                    decrease_count = 0
                    increase_streak += 1
                elif rougeL_score < max_rougeL_so_far:
                    decrease_count += 1
                    no_improve_count += 1
                    increase_streak = 0
                else:
                    no_improve_count += 1
                    increase_streak = 0

                if no_improve_count >= N or decrease_count >= K:
                    if max_rougeL_so_far >= 0.7:
                        data_prompt.append(best_summarizer_prompt)
                    print("Stopping early due to no improvement or consistent drop.")
                    break

                if rougeL_score < self.threshold:
                    self.summarizer_prompt = self.teacher_agent.run(
                        extracted_text=extracted_text,
                        description=description,
                        generated_about=about,
                        rouge_score=rougeL_score,
                        summarizer_prompt=self.summarizer_prompt,
                    )
                else:
                    data_prompt.append(best_summarizer_prompt)
                    break

                print(f"New Summarizer Prompt: {self.summarizer_prompt}")

                iter += 1

            print(f"Best RougeL Score for Data #{i}: {best_score}")

            iteration_debug.append(
                {
                    "description_html_clean": html,
                    "description_short": description,
                    "iteration_debug": data_debug,
                    "best_ROUGE-L": best_score,
                    "best_summarizer_prompt": best_summarizer_prompt,
                }
            )

        self.summarizer_prompt = self.prompt_combine.run(prompt_list=data_prompt)

        print(f"Final Summarizer Prompt: {self.summarizer_prompt}")
        print(f"Number of disqualified samples: {len(train_data) - len(data_prompt)}")

        self.debug_result["Train Debug"] = iteration_debug
        self.debug_result["Final Extractor Prompt"] = self.extractor_prompt
        self.debug_result["Final Summarizer Prompt"] = self.summarizer_prompt

        with open("final_prompt.txt", "w") as f:
            f.write(self.summarizer_prompt)

        import json
        with open("debug_result.json", "w") as f:
            json.dump(self.debug_result, f, indent=2, ensure_ascii=False)

        print("final_prompt.txt and debug_result.json saved.")

# Login Huggingface

In [12]:
from huggingface_hub import login

login("your_huggingface_token")

# Run flow

In [13]:
import pandas as pd

df = pd.read_csv("/kaggle/input/train_data.csv")

train_data = df[["description_short", "description_html_clean"]].to_dict(orient="records")

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from string import Template
import re

shared_llm = LocalLLMClient(model_name="google/gemma-2-2b-it")

extractor_agent = ExtractorAgent(llm=shared_llm)
summarizer_agent = SummarizerAgent(llm=shared_llm)
teacher_agent = TeacherAgent(llm=shared_llm)
prompt_combine_agent = PromptCombineAgent(llm=shared_llm)

optimizer = ParallelOptimizer(
    extractor_agent=extractor_agent,
    summarizer_agent=summarizer_agent,
    teacher_agent=teacher_agent,
    prompt_combine_agent=prompt_combine_agent,
    threshold=0.7
)
optimizer.run(max_iterations=15, train_data=train_data)

2025-08-09 04:23:21.701527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754713401.727213     233 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754713401.734682     233 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Data #0:
- Description: The KTXS News app delivers news, weather and sports in an instant.


W0809 04:24:09.196000 233 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
W0809 04:24:09.206000 233 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=1)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Teacher raw answer:
 ```
Generate a short, concise description for the following app, focusing on its main purpose:

<EXTRACTED_TEXT>
$extracted_text
```
New Summarizer Prompt: ```
Generate a short, concise description for the following app, focusing on its main purpose:

<EXTRACTED_TEXT>
$extracted_text
```
Iteration #2:
Extracted Text: This app is a collection of Jung Jaehyun wallpapers for KPOP fans.
Generated About: Jung Jaehyun wallpapers for KPOP fans.
Rouge1 Score: 0.625
Rouge2 Score: 0.2857142857142857
RougeL Score: 0.625


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Teacher raw answer:
 ```
Generate a short, concise description for the following app, focusing on its main purpose and highlighting its key features.  The description should be informative and engaging. 

<EXTRACTED_TEXT>
$extracted_text
```
New Summarizer Prompt: ```
Generate a short, concise description for the following app, focusing on its main purpose and highlighting its key features.  The description should be informative and engaging. 

<EXTRACTED_TEXT>
$extracted_text
```
Iteration #3:
Extracted Text: This app is a collection of Jung Jaehyun wallpapers for KPOP fans.
Generated About: ##  Immerse Yourself in Jung Jaehyun's World with Stunning Wallpapers 

**This app is your ultimate destination for
Rouge1 Score: 0.29629629629629634
Rouge2 Score: 0.08
RougeL Score: 0.29629629629629634


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Teacher raw answer:
 ```
Generate a short, concise description for the following app. The description should be informative and engaging, focusing on its main purpose and highlighting key features.  

<EXTRACTED_TEXT>
$extracted_text
```

**Explanation of Changes:**

* **Removed explicit ROUGE score mention:**  We don't need to mention the ROUGE score in the prompt as it's likely not useful for the LLM to know how it's performing. 
* **Explicitly prioritize key features:** The prompt now explicitly states to focus on the app's "main
New Summarizer Prompt: ```
Generate a short, concise description for the following app. The description should be informative and engaging, focusing on its main purpose and highlighting key features.  

<EXTRACTED_TEXT>
$extracted_text
```

**Explanation of Changes:**

* **Removed explicit ROUGE score mention:**  We don't need to mention the ROUGE score in the prompt as it's likely not useful for the LLM to know how it's performing. 
* **Explicitly prioriti